<a href="https://colab.research.google.com/github/wallas999/Mis_Proyectos/blob/main/Elevators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Version 1.2.2
# [ES] 1 elevador; 2 lógicas de operación: FCFS & LOOK; todos al mismo destino: 1er piso. Además se agrega Timestamp y Tiempo de Espera
# [EN] 1 elevator, 2 operation logics: FCFS & LOOK; all same destination: 1st floor. Additionally, Timestamp and Waiting Time are added.

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import HTML
from matplotlib import gridspec
from matplotlib import animation



In [25]:
def simular_fcfs(
    llamadas,
    piso_inicial,
    consumo_subida=4.0,
    consumo_bajada=1.0,
    consumo_parada=1.0,
    tiempo_por_piso=5,
    tiempo_parada=10
):
    piso_actual = piso_inicial
    consumo_total = 0.0
    tiempo_total = 0
    historial = []

    def mover_ascensor(desde, hasta, es_recogida=False):
        nonlocal consumo_total, tiempo_total
        if desde == hasta:
            return
        sube = hasta > desde
        pisos_mov = abs(hasta - desde)
        energia_mov = pisos_mov * (consumo_subida if sube else consumo_bajada)
        energia_etapa = energia_mov + consumo_parada
        tiempo_etapa = pisos_mov * tiempo_por_piso + tiempo_parada
        consumo_total += energia_etapa
        tiempo_total += tiempo_etapa
        historial.append({
            "Timestamp (s)": tiempo_total,
            "Desde": desde, "Hasta": hasta, "Sube": sube,
            "Distancia (pisos)": pisos_mov, "Energía etapa (Wh)": energia_etapa,
            "Energía acumulada (Wh)": consumo_total, "Tiempo etapa (s)": tiempo_etapa,
            "Tiempo acumulado (s)": tiempo_total,
            "Tiempo espera (s)": tiempo_total if es_recogida else None
        })

    for origen, destino in llamadas:
        mover_ascensor(piso_actual, origen, es_recogida=True)  # Recogida = tiempo espera
        mover_ascensor(origen, destino, es_recogida=False)
        piso_actual = destino

    return pd.DataFrame(historial), consumo_total, tiempo_total


def simular_look(
    llamadas,
    piso_inicial,
    consumo_subida=4.0,
    consumo_bajada=1.0,
    consumo_parada=1.0,
    tiempo_por_piso=5,
    tiempo_parada=10
):
    piso_actual = piso_inicial
    consumo_total = 0.0
    tiempo_total = 0
    historial = []

    def mover_ascensor(desde, hasta, es_recogida=False):
        nonlocal consumo_total, tiempo_total
        if desde == hasta:
            return
        sube = hasta > desde
        pisos_mov = abs(hasta - desde)
        energia_mov = pisos_mov * (consumo_subida if sube else consumo_bajada)
        energia_etapa = energia_mov + consumo_parada
        tiempo_etapa = pisos_mov * tiempo_por_piso + tiempo_parada
        consumo_total += energia_etapa
        tiempo_total += tiempo_etapa
        historial.append({
            "Timestamp (s)": tiempo_total,
            "Desde": desde, "Hasta": hasta, "Sube": sube,
            "Distancia (pisos)": pisos_mov, "Energía etapa (Wh)": energia_etapa,
            "Energía acumulada (Wh)": consumo_total, "Tiempo etapa (s)": tiempo_etapa,
            "Tiempo acumulado (s)": tiempo_total,
            "Tiempo espera (s)": tiempo_total if es_recogida else None
        })

    llamadas_arriba = [(o, d) for o, d in llamadas if d > o]
    llamadas_arriba.sort(key=lambda x: x[0])

    llamadas_abajo = [(o, d) for o, d in llamadas if d < o]
    llamadas_abajo.sort(key=lambda x: x[0], reverse=True)

    if llamadas_arriba:
        for origen, _ in llamadas_arriba:
            mover_ascensor(piso_actual, origen, es_recogida=True)  # Recogida
            piso_actual = origen
        destinos_subida = sorted(set(d for _, d in llamadas_arriba))
        for destino in destinos_subida:
            mover_ascensor(piso_actual, destino, es_recogida=False)
            piso_actual = destino

    if llamadas_abajo:
        for origen, _ in llamadas_abajo:
            mover_ascensor(piso_actual, origen, es_recogida=True)  # Recogida
            piso_actual = origen
        destinos_bajada = sorted(set(d for _, d in llamadas_abajo), reverse=True)
        for destino in destinos_bajada:
            mover_ascensor(piso_actual, destino, es_recogida=False)
            piso_actual = destino

    return pd.DataFrame(historial), consumo_total, tiempo_total


In [26]:
# --- Bloque 3: Datos de prueba, ejecución de simulaciones y animación ---

# Parámetros de prueba
piso_inicial = 3
llamadas = [(2, 1), (4, 1), (6, 1)]  # primera llamada cambiada

# Ejecutar simulaciones
df_fcfs, energia_fcfs, tiempo_fcfs = simular_fcfs(llamadas, piso_inicial)
df_look, energia_look, tiempo_look = simular_look(llamadas, piso_inicial)

# Recorridos de pisos para FCFS
pisos_recorrido_fcfs = []
for i, row in df_fcfs.iterrows():
    pisos_recorrido_fcfs.append(row["Desde"])
    pisos_recorrido_fcfs.append(row["Hasta"])

# Recorridos de pisos para LOOK
pisos_recorrido_look = []
for i, row in df_look.iterrows():
    pisos_recorrido_look.append(row["Desde"])
    pisos_recorrido_look.append(row["Hasta"])

piso_min = min(min(pisos_recorrido_fcfs), min(pisos_recorrido_look))
piso_max = max(max(pisos_recorrido_fcfs), max(pisos_recorrido_look))

plt.ioff()
fig = plt.figure(figsize=(14, 6))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.8])

# Animación FCFS (columna 0)
ax_fcfs = fig.add_subplot(gs[0])
ax_fcfs.set_ylim(piso_min - 1, piso_max + 1)
ax_fcfs.set_xlim(0, 1)
ax_fcfs.set_xticks([])
ax_fcfs.set_ylabel("Piso")
ax_fcfs.set_title("Elevator FCFS")
for piso in range(piso_min, piso_max + 1):
    ax_fcfs.hlines(piso, 0, 1, colors='lightgray', linestyles='dashed')
punto_fcfs, = ax_fcfs.plot([], [], 's', markersize=20, color='blue')

# Animación LOOK (columna 1)
ax_look = fig.add_subplot(gs[1], sharey=ax_fcfs)
ax_look.set_ylim(piso_min - 1, piso_max + 1)
ax_look.set_xlim(0, 1)
ax_look.set_xticks([])
ax_look.set_title("Elevator LOOK")
for piso in range(piso_min, piso_max + 1):
    ax_look.hlines(piso, 0, 1, colors='lightgray', linestyles='dashed')
punto_look, = ax_look.plot([], [], 's', markersize=20, color='green')

# Gráfico energía y tiempo (columna 2)
ax_bar = fig.add_subplot(gs[2])
ax_bar.bar(["FCFS", "LOOK"], [energia_fcfs, energia_look], color=["skyblue", "lightgreen"], alpha=0.7)
ax_bar.set_ylabel("Energía (Wh)")
ax_bar.set_title("Comparación Energía y Tiempo")

ax_time = ax_bar.twinx()
ax_time.plot(["FCFS", "LOOK"], [tiempo_fcfs, tiempo_look], color="red", marker="o", label="Tiempo (s)")
ax_time.set_ylabel("Tiempo (s)")
ax_time.legend(loc="upper right")

max_frames = max(len(pisos_recorrido_fcfs), len(pisos_recorrido_look))

def init():
    punto_fcfs.set_data([], [])
    punto_look.set_data([], [])
    return punto_fcfs, punto_look

def actualizar(frame):
    if frame < len(pisos_recorrido_fcfs):
        piso_f = pisos_recorrido_fcfs[frame]
        punto_fcfs.set_data([0.5], [piso_f])
    else:
        punto_fcfs.set_data([0.5], [pisos_recorrido_fcfs[-1]])

    if frame < len(pisos_recorrido_look):
        piso_l = pisos_recorrido_look[frame]
        punto_look.set_data([0.5], [piso_l])
    else:
        punto_look.set_data([0.5], [pisos_recorrido_look[-1]])

    return punto_fcfs, punto_look

ani = animation.FuncAnimation(
    fig, actualizar, frames=max_frames,
    init_func=init, blit=True, interval=1000, repeat=False
)

HTML(ani.to_jshtml())
